In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor, Pool, metrics, cv

In [2]:
test_df = pd.read_csv('data/test.csv')
train_df = pd.read_csv('data/train.csv')
sample_submission_df = pd.read_csv('data/sample_submission.csv')

In [3]:
y = train_df.SalePrice
X = train_df.drop(['SalePrice', 'Id'], axis=1)

In [4]:
submission_id = test_df.Id
final_test_df = test_df.drop(['Id'], axis=1)

In [5]:
print(X.shape, final_test_df.shape)

(1460, 79) (1459, 79)


In [6]:
df = pd.concat([X, final_test_df])

In [7]:
df.shape

(2919, 79)

In [8]:
df.fillna({'LotFrontage': X.LotFrontage.median()}, inplace=True)
df.fillna(0, inplace=True)

In [9]:
X = df.iloc[:1460]
final_test_df = df.iloc[1460:]

In [10]:
print(X.shape, final_test_df.shape)

(1460, 79) (1459, 79)


In [11]:
X.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object

In [12]:
categorical_features_indices = np.where((X.dtypes != 'float64') & (X.dtypes != 'int64'))[0]

In [13]:
categorical_features_indices

array([ 1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 20, 21, 22, 23,
       24, 26, 27, 28, 29, 30, 31, 32, 34, 38, 39, 40, 41, 52, 54, 56, 57,
       59, 62, 63, 64, 71, 72, 73, 77, 78], dtype=int64)

In [14]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=0)

In [18]:
model = CatBoostRegressor(
    random_seed=42,
    iterations=1500
)

In [19]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_validation, y_validation),
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.040334
0:	learn: 76687.7474163	test: 79123.7638133	best: 79123.7638133 (0)	total: 29.2ms	remaining: 43.8s
1:	learn: 74695.9056002	test: 76999.4155617	best: 76999.4155617 (1)	total: 58.7ms	remaining: 44s
2:	learn: 72848.6680597	test: 75188.3044079	best: 75188.3044079 (2)	total: 88.1ms	remaining: 44s
3:	learn: 71165.6617299	test: 73731.3476119	best: 73731.3476119 (3)	total: 122ms	remaining: 45.5s
4:	learn: 69572.1930161	test: 72001.1360970	best: 72001.1360970 (4)	total: 155ms	remaining: 46.5s
5:	learn: 67833.1518440	test: 70184.1305075	best: 70184.1305075 (5)	total: 188ms	remaining: 46.9s
6:	learn: 66183.0466759	test: 68556.3065637	best: 68556.3065637 (6)	total: 222ms	remaining: 47.3s
7:	learn: 64745.8249236	test: 67217.0344249	best: 67217.0344249 (7)	total: 255ms	remaining: 47.6s
8:	learn: 63442.3422725	test: 65885.2690971	best: 65885.2690971 (8)	total: 272ms	remaining: 45s
9:	learn: 62108.8917989	test: 64646.6543253	best: 64646.6543253 (9)	total: 304ms	remaining:

87:	learn: 24062.3772119	test: 31516.3789694	best: 31516.3789694 (87)	total: 3.49s	remaining: 56.1s
88:	learn: 23916.2519350	test: 31480.1740191	best: 31480.1740191 (88)	total: 3.53s	remaining: 56s
89:	learn: 23796.1414198	test: 31448.7114153	best: 31448.7114153 (89)	total: 3.56s	remaining: 55.8s
90:	learn: 23659.2124337	test: 31362.5289187	best: 31362.5289187 (90)	total: 3.6s	remaining: 55.7s
91:	learn: 23558.3980087	test: 31293.1872494	best: 31293.1872494 (91)	total: 3.64s	remaining: 55.7s
92:	learn: 23434.2510494	test: 31176.6832358	best: 31176.6832358 (92)	total: 3.67s	remaining: 55.5s
93:	learn: 23325.6158372	test: 31145.3801734	best: 31145.3801734 (93)	total: 3.71s	remaining: 55.4s
94:	learn: 23211.1300670	test: 31096.4699981	best: 31096.4699981 (94)	total: 3.75s	remaining: 55.5s
95:	learn: 23118.6471213	test: 31035.2215389	best: 31035.2215389 (95)	total: 3.79s	remaining: 55.4s
96:	learn: 22960.4001509	test: 30876.7744267	best: 30876.7744267 (96)	total: 3.83s	remaining: 55.4s
97:

169:	learn: 18553.5727881	test: 29181.4111987	best: 29181.4111987 (169)	total: 6.53s	remaining: 51.1s
170:	learn: 18533.2201203	test: 29202.3176693	best: 29181.4111987 (169)	total: 6.57s	remaining: 51.1s
171:	learn: 18492.2766899	test: 29165.5791197	best: 29165.5791197 (171)	total: 6.61s	remaining: 51s
172:	learn: 18460.3724173	test: 29136.7560538	best: 29136.7560538 (172)	total: 6.64s	remaining: 51s
173:	learn: 18456.7187490	test: 29116.8373134	best: 29116.8373134 (173)	total: 6.65s	remaining: 50.7s
174:	learn: 18410.9199498	test: 29101.5416331	best: 29101.5416331 (174)	total: 6.68s	remaining: 50.6s
175:	learn: 18344.7426884	test: 29055.6985101	best: 29055.6985101 (175)	total: 6.72s	remaining: 50.5s
176:	learn: 18296.9571325	test: 29051.3739163	best: 29051.3739163 (176)	total: 6.75s	remaining: 50.5s
177:	learn: 18239.9063132	test: 29038.1165087	best: 29038.1165087 (177)	total: 6.78s	remaining: 50.4s
178:	learn: 18209.5493168	test: 28989.2661461	best: 28989.2661461 (178)	total: 6.82s	r

255:	learn: 15841.7134808	test: 28422.9261679	best: 28406.2161297 (250)	total: 9.63s	remaining: 46.8s
256:	learn: 15816.5026350	test: 28419.8616726	best: 28406.2161297 (250)	total: 9.66s	remaining: 46.7s
257:	learn: 15797.3537984	test: 28431.9704481	best: 28406.2161297 (250)	total: 9.7s	remaining: 46.7s
258:	learn: 15759.8100777	test: 28438.3907547	best: 28406.2161297 (250)	total: 9.74s	remaining: 46.7s
259:	learn: 15722.2137777	test: 28433.1329282	best: 28406.2161297 (250)	total: 9.77s	remaining: 46.6s
260:	learn: 15697.0415173	test: 28411.9232665	best: 28406.2161297 (250)	total: 9.81s	remaining: 46.6s
261:	learn: 15660.2645338	test: 28407.6873509	best: 28406.2161297 (250)	total: 9.84s	remaining: 46.5s
262:	learn: 15645.2317916	test: 28409.3607201	best: 28406.2161297 (250)	total: 9.89s	remaining: 46.5s
263:	learn: 15619.9551755	test: 28407.7531235	best: 28406.2161297 (250)	total: 9.93s	remaining: 46.5s
264:	learn: 15607.4036632	test: 28406.0289730	best: 28406.0289730 (264)	total: 9.97

...

1094:	learn: 7391.8288434	test: 26930.1671706	best: 26930.1671706 (1094)	total: 40.5s	remaining: 15s
1095:	learn: 7386.9060316	test: 26927.4251923	best: 26927.4251923 (1095)	total: 40.5s	remaining: 14.9s
1096:	learn: 7382.6012912	test: 26932.4850358	best: 26927.4251923 (1095)	total: 40.5s	remaining: 14.9s
1097:	learn: 7374.4149311	test: 26928.7193579	best: 26927.4251923 (1095)	total: 40.6s	remaining: 14.9s
1098:	learn: 7370.0814560	test: 26928.4641608	best: 26927.4251923 (1095)	total: 40.6s	remaining: 14.8s
1099:	learn: 7360.9308349	test: 26926.5563943	best: 26926.5563943 (1099)	total: 40.7s	remaining: 14.8s
1100:	learn: 7350.4713394	test: 26920.0234226	best: 26920.0234226 (1100)	total: 40.7s	remaining: 14.7s
1101:	learn: 7344.3919284	test: 26917.3391362	best: 26917.3391362 (1101)	total: 40.7s	remaining: 14.7s
1102:	learn: 7338.5948951	test: 26918.0008853	best: 26917.3391362 (1101)	total: 40.8s	remaining: 14.7s
1103:	learn: 7334.7137132	test: 26912.1374905	best: 26912.1374905 (1103)	to

1179:	learn: 6925.6002009	test: 26850.2249068	best: 26845.2172048 (1176)	total: 43.5s	remaining: 11.8s
1180:	learn: 6914.6077736	test: 26844.1746703	best: 26844.1746703 (1180)	total: 43.6s	remaining: 11.8s
1181:	learn: 6912.5165835	test: 26843.7998161	best: 26843.7998161 (1181)	total: 43.6s	remaining: 11.7s
1182:	learn: 6908.5352909	test: 26843.2520479	best: 26843.2520479 (1182)	total: 43.7s	remaining: 11.7s
1183:	learn: 6904.4870782	test: 26845.2618137	best: 26843.2520479 (1182)	total: 43.7s	remaining: 11.7s
1184:	learn: 6903.7211089	test: 26845.0841969	best: 26843.2520479 (1182)	total: 43.8s	remaining: 11.6s
1185:	learn: 6898.2288699	test: 26847.7249151	best: 26843.2520479 (1182)	total: 43.8s	remaining: 11.6s
1186:	learn: 6896.4817259	test: 26847.5386482	best: 26843.2520479 (1182)	total: 43.8s	remaining: 11.6s
1187:	learn: 6893.4382797	test: 26847.8450101	best: 26843.2520479 (1182)	total: 43.9s	remaining: 11.5s
1188:	learn: 6892.7687124	test: 26847.6122103	best: 26843.2520479 (1182)	

1264:	learn: 6596.8303136	test: 26818.6014637	best: 26816.3313682 (1249)	total: 46.7s	remaining: 8.68s
1265:	learn: 6596.1034743	test: 26818.7684304	best: 26816.3313682 (1249)	total: 46.7s	remaining: 8.64s
1266:	learn: 6595.3368608	test: 26821.1173911	best: 26816.3313682 (1249)	total: 46.8s	remaining: 8.6s
1267:	learn: 6594.3810400	test: 26820.0440706	best: 26816.3313682 (1249)	total: 46.8s	remaining: 8.56s
1268:	learn: 6588.8338939	test: 26820.1243678	best: 26816.3313682 (1249)	total: 46.8s	remaining: 8.53s
1269:	learn: 6586.0455365	test: 26819.3765651	best: 26816.3313682 (1249)	total: 46.9s	remaining: 8.49s
1270:	learn: 6581.1061164	test: 26820.3144821	best: 26816.3313682 (1249)	total: 46.9s	remaining: 8.45s
1271:	learn: 6568.5777394	test: 26814.5729805	best: 26814.5729805 (1271)	total: 47s	remaining: 8.42s
1272:	learn: 6567.9574241	test: 26814.4822663	best: 26814.4822663 (1272)	total: 47s	remaining: 8.38s
1273:	learn: 6560.0213625	test: 26811.3109051	best: 26811.3109051 (1273)	total

1348:	learn: 6316.1207869	test: 26781.3077340	best: 26779.7018811 (1337)	total: 50s	remaining: 5.6s
1349:	learn: 6315.5632032	test: 26781.2197102	best: 26779.7018811 (1337)	total: 50.1s	remaining: 5.56s
1350:	learn: 6306.0355698	test: 26780.7158770	best: 26779.7018811 (1337)	total: 50.1s	remaining: 5.53s
1351:	learn: 6302.0093095	test: 26780.2038027	best: 26779.7018811 (1337)	total: 50.1s	remaining: 5.49s
1352:	learn: 6301.8880893	test: 26780.1999105	best: 26779.7018811 (1337)	total: 50.2s	remaining: 5.45s
1353:	learn: 6297.9784560	test: 26779.1767772	best: 26779.1767772 (1353)	total: 50.2s	remaining: 5.42s
1354:	learn: 6292.9584273	test: 26782.3493707	best: 26779.1767772 (1353)	total: 50.3s	remaining: 5.38s
1355:	learn: 6288.8690247	test: 26782.8440681	best: 26779.1767772 (1353)	total: 50.3s	remaining: 5.34s
1356:	learn: 6283.8338260	test: 26782.6190334	best: 26779.1767772 (1353)	total: 50.3s	remaining: 5.3s
1357:	learn: 6280.8321073	test: 26782.4025368	best: 26779.1767772 (1353)	tota

1429:	learn: 6037.8644601	test: 26756.9492815	best: 26756.0910851 (1427)	total: 52.9s	remaining: 2.59s
1430:	learn: 6034.3369600	test: 26755.9554844	best: 26755.9554844 (1430)	total: 53s	remaining: 2.55s
1431:	learn: 6033.2045608	test: 26757.1684222	best: 26755.9554844 (1430)	total: 53s	remaining: 2.52s
1432:	learn: 6031.4632033	test: 26756.1566141	best: 26755.9554844 (1430)	total: 53.1s	remaining: 2.48s
1433:	learn: 6024.0679540	test: 26755.7161911	best: 26755.7161911 (1433)	total: 53.1s	remaining: 2.44s
1434:	learn: 6020.8679903	test: 26757.7697162	best: 26755.7161911 (1433)	total: 53.1s	remaining: 2.41s
1435:	learn: 6017.4975792	test: 26756.5499740	best: 26755.7161911 (1433)	total: 53.2s	remaining: 2.37s
1436:	learn: 6012.9190499	test: 26755.2030437	best: 26755.2030437 (1436)	total: 53.2s	remaining: 2.33s
1437:	learn: 6005.2114767	test: 26752.3516255	best: 26752.3516255 (1437)	total: 53.2s	remaining: 2.29s
1438:	learn: 6001.6268390	test: 26755.2432390	best: 26752.3516255 (1437)	tota

In [23]:
y_pred = model.predict(final_test_df)

In [24]:
submission = pd.DataFrame({'Id': submission_id, 'SalePrice': y_pred})

In [25]:
submission.to_csv('CatBoost.csv',index=False)